In [1]:
import numpy as np

import pandas as pd
import pandas_ta as ta # pandas technical analysis
import pandas_datareader.data as web

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,6)
plt.style.use("classic") # plt.style.available[:] gives list of all available stylesheets
plt.rcParams["font.size"] = 11
plt.rcParams["lines.linestyle"] = "-"
plt.rcParams["figure.dpi"] = 700 #sharp image

import seaborn as sns

from datetime import datetime

import yfinance as yf

from ta.momentum import RSIIndicator
from ta.momentum import ROCIndicator
from ta.momentum import StochasticOscillator
from ta.momentum import WilliamsRIndicator

from ta.trend import MACD

from ta.volatility import BollingerBands

from ta.volume import OnBalanceVolumeIndicator

In [2]:
end = datetime.today()
start = datetime(end.year-1, end.month, end.day)

ticker = ["^GSPC", 
          "^IXIC"
         ] # add tickers here

df = web.DataReader(ticker, "yahoo", start, end)

In [3]:
# .stack() converts the innermost column index into the innermost row index
df = df.stack() # https://medium.com/swlh/reshaping-in-pandas-with-stack-and-unstack-functions-bb169f64467d
df = df.reset_index() # not necessary, puts the same date to each innermost row index
df

Attributes,Date,Symbols,Adj Close,Close,High,Low,Open,Volume
0,2020-09-15,^GSPC,3401.199951,3401.199951,3419.479980,3389.250000,3407.729980,4051460000
1,2020-09-15,^IXIC,11190.320312,11190.320312,11244.459961,11127.980469,11193.959961,3772820000
2,2020-09-16,^GSPC,3385.489990,3385.489990,3428.919922,3384.449951,3411.229980,4710030000
3,2020-09-16,^IXIC,11050.469727,11050.469727,11245.419922,11046.429688,11222.080078,3661710000
4,2020-09-17,^GSPC,3357.010010,3357.010010,3375.169922,3328.820068,3346.860107,4371940000
...,...,...,...,...,...,...,...,...
499,2021-09-10,^IXIC,15115.490234,15115.490234,15349.469727,15111.309570,15332.919922,4567980000
500,2021-09-13,^GSPC,4468.729980,4468.729980,4492.990234,4445.700195,4474.810059,3096390000
501,2021-09-13,^IXIC,15105.580078,15105.580078,15215.440430,15030.849609,15211.429688,4701190000
502,2021-09-14,^GSPC,4443.049805,4443.049805,4485.680176,4435.459961,4479.330078,2568730000


In [6]:
class TechnicalIndicators:
    """
    This is an example of the TechIndicator class.
    """
    def __init__(self, symbol, close, high, low, volume):
        self.symbol = symbol
        self.close = close
        self.high = high
        self.low = low
        self.volume = volume
        self.technical_indicators = None
        
    def set_technical_indicators(self):
        self.technical_indicators = pd.DataFrame()
        
        self.technical_indicators["Close"] = self.close
        self.technical_indicators["High"] = self.high
        self.technical_indicators["Low"] = self.low
        self.technical_indicators["Volume"] = self.volume

# momentum indicators
    def rate_change(self, window):
        """
        Rate of Change. Momentum Indicator
        """
        closing_price = self.close
        dataframe = self.technical_indicators
        
        roc = ROCIndicator(close = closing_price,
                    window = window,
                    fillna = False
                    )
        
        dataframe["ROC"] = roc.roc()

        return dataframe    
    
    def rsi(self, window):
        """
        The relative strength index. Momentum Indicator.
        """
        closing_price = self.close
        dataframe = self.technical_indicators
        
        rsi = RSIIndicator(close = closing_price, 
                           window = window,
                           fillna = False
                          )
        
        dataframe["RSI"] = rsi.rsi()
        
        return dataframe
    
    def stochastic_oscillator(self, window):
        """
        Stochastic Oscillator. Momentum Indicator.
        """
        dataframe = self.technical_indicators
        
        stoch_osci = StochasticOscillator(
            close = self.close,
            high = self.high,
            low = self.low,
            window = window,
            smooth_window = 3,
            fillna = False
             )
        
        dataframe["Stochastic Oscillator"] = stoch_osci.stoch()
    
        return dataframe
    
    def WilliamsR(self, window):
        """
        WilliamsR Indicator. Momentum Indicator.
        """
        dataframe = self.technical_indicators

        williamsR = WilliamsRIndicator(high = self.high,
                                      low = self.low,
                                      close = self.close,
                                      lbp = window,
                                      fillna = False
                                      )
    
        dataframe["Williams %R Indicator"] = williamsR.williams_r()
    
        return dataframe
    
# trend indicators
    def moving_average(self):
        """
        Moving Average.
        """
        dataframe = self.technical_indicators
        days = [5,10,20]
        
        for i in days:
            col_name = f"MA for {i} days"
            dataframe[col_name] = self.price.rolling(window = i).mean()
            
        return dataframe
    
    def macd(self):
        """
        The moving average convergence/divergence.
        """
        dataframe = self.technical_indicators
        closing_price = self.close
        ema_26 = 26
        ema_12 = 12
        signal = 9
        macd = MACD(closing_price, 
                    window_fast = ema_12, 
                    window_slow = ema_26,
                    window_sign = signal,
                    fillna = False
                    )
        dataframe["MACD"] = macd.macd()
        
        return dataframe

# volatility indicators
    def bollinger_bands(self, window):
        """
        Bollinger bands. Volatility Indicator.
        """
        closing_price = self.close
        dataframe = self.technical_indicators
        
        bbands = BollingerBands(close = closing_price,
                                window = window,
                                window_dev = 2,
                                fillna= False
                               )
        
        dataframe["Bollinger Bands Middle"] = bbands.bollinger_mavg()
        dataframe["Bollinger Bands Lower"] = bbands.bollinger_hband()
        dataframe["Bollinger Bands Upper"] = bbands.bollinger_lband()
        
        return dataframe
    
# volume indicators
    def on_balance_volume(self):
        """On Balance Volume. Volume Indicator."""
        dataframe = self.technical_indicators
        
        on_bal_vol = OnBalanceVolumeIndicator(close = self.close,
                                  volume = self.volume,
                                  fillna = False
                                  )
        
        dataframe["On Balance Volume"] = on_bal_vol.on_balance_volume()
        
        return dataframe

In [7]:
gspc = df[df["Symbols"] == "^GSPC"]
ixic = df[df["Symbols"] == "^IXIC"]

symbol = TechnicalIndicators(gspc, gspc["Close"], gspc["High"], gspc["Low"], gspc["Volume"])
symbol.set_technical_indicators()
symbol.rsi(14)
symbol.bollinger_bands(20)
symbol.rate_change(30).tail(40)
symbol.rate_change(1)
symbol.stochastic_oscillator(14)
symbol.WilliamsR(14)
symbol.on_balance_volume()

,Close,High,Low,Volume,RSI,Bollinger Bands Middle,Bollinger Bands Lower,Bollinger Bands Upper,ROC,Stochastic Oscillator,Williams %R Indicator,On Balance Volume
0,3401.199951,3419.479980,3389.250000,4051460000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4051460000
2,3385.489990,3428.919922,3384.449951,4710030000,NaN,NaN,NaN,NaN,-0.461895,NaN,NaN,-658570000
4,3357.010010,3375.169922,3328.820068,4371940000,NaN,NaN,NaN,NaN,-0.841237,NaN,NaN,-5030510000
6,3319.469971,3362.270020,3292.399902,7068700000,NaN,NaN,NaN,NaN,-1.118258,NaN,NaN,-12099210000
8,3281.060059,3285.570068,3229.100098,4828350000,NaN,NaN,NaN,NaN,-1.157110,NaN,NaN,-16927560000
...,...,...,...,...,...,...,...,...,...,...,...,...
494,4514.069824,4521.790039,4493.950195,2808480000,60.668592,4483.167993,4563.930503,4402.405483,-0.131857,82.157954,-17.842046,88638960000
496,4493.279785,4529.899902,4492.069824,3035300000,55.223401,4485.711475,4564.355707,4407.067242,-0.460561,62.193311,-37.806689,85603660000
498,4458.580078,4520.470215,4457.660156,2851140000,47.552024,4485.598975,4564.391553,4406.806396,-0.772258,8.675213,-91.324787,82752520000
500,4468.729980,4492.990234,4445.700195,3096390000,49.750893,4485.635474,4564.396079,4406.874869,0.227649,22.995315,-77.004685,85848910000
